In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, vstack, Column, join
from astropy import units as u
from astroquery.simbad import Simbad
from astroquery.gaia import Gaia
import time

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [2]:
sample = Table.read('Gr8stars_GaiaeDR3_TIC_hires.csv')
sample.sort(['_RAJ2000','_DEJ2000'])

In [3]:
table = sample['_RAJ2000', '_DEJ2000']
table.rename_column('_RAJ2000', 'ra_j2000')
table.rename_column('_DEJ2000', 'dec_j2000')

In [4]:
# names:
table['primary_name'] = sample['EDR3Name'] # CHANGE THIS LATER
table['gaiadr2_source_id'] = sample['GAIA']
table['gaiaedr3_source_id'] = sample['Source']
table['hip'] = sample['HIP']
#table['hip'][table['hip'] < 0] = np.nan # replace -1s with nans
table.add_column(Column(name='hd', dtype='<U29', length=len(table)))
table['tic'] = sample['TIC']
table['2mass'] = sample['2MASS']
table['2mass'][table['2mass']=='--'] = '' # replace --s with empty
table['wisea'] = sample['WISEA']
table['wisea'][table['wisea']=='--'] = '' # replace --s with empty


In [5]:
# gaia:
table['ra'] = sample['RA_ICRS']
table['ra_error'] = sample['e_RA_ICRS']
table['dec'] = sample['DE_ICRS']
table['dec_error'] = sample['e_DE_ICRS']
table['parallax'] = sample['Plx']
table['parallax_error'] = sample['e_Plx']
table['pmra'] = sample['pmRA']
table['pmra_error'] = sample['e_pmRA']
table['pmdec'] = sample['pmDE']
table['pmdec_error'] = sample['e_pmDE']
table['gmag'] = sample['Gmag']
table['bp_rp'] = sample['BPRP']
table['radial_velocity'] = sample['RVDR2']
table['radial_velocity_error'] = sample['e_RVDR2']
table['ref_epoch'] = 'J2016.0'

In [6]:
# hacky alternative to rerunning all the simbad code:
table2 = Table.read('Gr8stars_GaiaeDR3_formatted-names.csv')
table2.sort(['ra_j2000','dec_j2000'])
table['hd'] = table2['hd']
table['primary_name'] = table2['primary_name']

In [7]:
# gaia DR2:
table['primary_name','gaiadr2_source_id'].write('gaiadr2.vot', format='votable', overwrite=True)  # write out the DR2 source_ids
#Gaia.ROW_LIMIT = 5000
query = "SELECT TOP 5000 * FROM tap_upload.tbl AS tbl, gaiadr2.gaia_source AS gaia WHERE tbl.gaiadr2_source_id=gaia.source_id"
j = Gaia.launch_job(query=query, upload_resource='gaiadr2.vot', 
                    upload_table_name="tbl", verbose=True)
r = j.get_results()

Launched query: 'SELECT TOP 5000 * FROM tap_upload.tbl AS tbl, gaiadr2.gaia_source AS gaia WHERE tbl.gaiadr2_source_id=gaia.source_id'
------>https
host = gea.esac.esa.int:443
context = /tap-server/tap/sync
Content-type = multipart/form-data; boundary====1621886453599===
200 200
[('Date', 'Mon, 24 May 2021 20:00:54 GMT'), ('Server', 'Apache/2.4.6 (CentOS) OpenSSL/1.0.2k-fips PHP/5.4.16 mod_jk/1.2.43'), ('Cache-Control', 'no-cache, no-store, max-age=0, must-revalidate'), ('Pragma', 'no-cache'), ('Expires', '0'), ('X-XSS-Protection', '1; mode=block'), ('X-Frame-Options', 'SAMEORIGIN'), ('X-Content-Type-Options', 'nosniff'), ('Set-Cookie', 'JSESSIONID=0FB7F8691B86EBDF2C8B76A4F67D32B8; Path=/tap-server; Secure; HttpOnly'), ('Content-Encoding', 'gzip'), ('Content-Disposition', 'attachment;filename="1621886454252O-result.vot"'), ('Transfer-Encoding', 'chunked'), ('Content-Type', 'application/x-votable+xml')]
Retrieving sync. results...
Query finished.


In [8]:
r.columns

<TableColumns names=('primary_name','gaiadr2_source_id','solution_id','designation','source_id','random_index','ref_epoch','ra','ra_error','dec','dec_error','parallax','parallax_error','parallax_over_error','pmra','pmra_error','pmdec','pmdec_error','ra_dec_corr','ra_parallax_corr','ra_pmra_corr','ra_pmdec_corr','dec_parallax_corr','dec_pmra_corr','dec_pmdec_corr','parallax_pmra_corr','parallax_pmdec_corr','pmra_pmdec_corr','astrometric_n_obs_al','astrometric_n_obs_ac','astrometric_n_good_obs_al','astrometric_n_bad_obs_al','astrometric_gof_al','astrometric_chi2_al','astrometric_excess_noise','astrometric_excess_noise_sig','astrometric_params_solved','astrometric_primary_flag','astrometric_weight_al','astrometric_pseudo_colour','astrometric_pseudo_colour_error','mean_varpi_factor_al','astrometric_matched_observations','visibility_periods_used','astrometric_sigma5d_max','frame_rotator_object_type','matched_observations','duplicated_source','phot_g_n_obs','phot_g_mean_flux','phot_g_mean_fl

In [9]:
dr2_table = r['gaiadr2_source_id', 'teff_val']
dr2_table.rename_column('teff_val', 'gaia_teff')
dr2_table['gaia_teff_lower'] = r['teff_percentile_lower']
dr2_table['gaia_teff_upper'] = r['teff_percentile_upper']
dr2_table['gaia_radius'] = r['radius_val']
dr2_table['gaia_radius_lower'] = r['radius_percentile_lower']
dr2_table['gaia_radius_upper'] = r['radius_percentile_upper']
dr2_table['gaia_lum'] = r['lum_val']
dr2_table['gaia_lum_lower'] = r['lum_percentile_lower']
dr2_table['gaia_lum_upper'] = r['lum_percentile_upper']

In [10]:
table_with_dr2 = join(table, dr2_table, keys='gaiadr2_source_id', join_type='left')

In [11]:
assert len(table_with_dr2) == len(table)

In [12]:
table_with_dr2.sort(['ra_j2000','dec_j2000']) # get it in the same order as sample
for i in np.random.randint(0,len(table),42):
    #print(table[i], table_with_dr2[i])
    assert table[i]['gaiadr2_source_id'] == table_with_dr2[i]['gaiadr2_source_id']

In [13]:
table = table_with_dr2

In [14]:
# spectra:
table.add_column(Column(name='spectrum_hires', dtype='<U29', length=len(table)))
table['spectrum_hires_resolution'] = sample['hires_maxres']
table['spectrum_hires_snr'] = sample['hires_maxres_snr']
table['spectrum_hires_nobs'] = sample['n_hires_obs_maxres_iodout']

In [15]:
# add a link:
MIN_SNR = 100 # if combined SNR is below this limit, ignore the spectra
table['spectrum_hires'][sample['hires_maxres_snr'] >= MIN_SNR] = 'true'
bad = sample['hires_maxres_snr'] < MIN_SNR
table['spectrum_hires_resolution'][bad] = 0
table['spectrum_hires_nobs'][bad] = 0
table['spectrum_hires_snr'][bad] = 0

In [16]:
table[0:10]

ra_j2000,dec_j2000,primary_name,gaiadr2_source_id,gaiaedr3_source_id,hip,hd,tic,2mass,wisea,ra,ra_error,dec,dec_error,parallax,parallax_error,pmra,pmra_error,pmdec,pmdec_error,gmag,bp_rp,radial_velocity,radial_velocity_error,ref_epoch,gaia_teff,gaia_teff_lower,gaia_teff_upper,gaia_radius,gaia_radius_lower,gaia_radius_upper,gaia_lum,gaia_lum_lower,gaia_lum_upper,spectrum_hires,spectrum_hires_resolution,spectrum_hires_snr,spectrum_hires_nobs
,,,,,,,,,,,,,,,,,,,,,,,,,K,K,K,Rsun,Rsun,Rsun,Lsun,Lsun,Lsun,,,,
float64,float64,str29,int64,int64,int64,str7,int64,str16,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str7,float32,float32,float32,float32,float32,float32,float32,float32,float32,str29,float64,float64,int64
0.1735187163099,62.1758995053047,HD224792,429915991435184000,429915991435184000,58,224792,417860263,00004167+6210331,J000041.58+621032.9,0.17307161161,0.0135,62.17570631925,0.0153,26.1048,0.0182,-46.955,0.017,-43.467,0.019,6.922204,0.67424,3.76,0.27,J2016.0,6208.75,6168.0,6261.6665,1.1794982,1.1596469,1.1951349,1.8625417,1.8590426,1.8660407,,0.0,0.0,0
0.2923327649201,-0.0761104076889,HD224839,2546019543207725440,2546019543207725440,93,224839,138684373,00011015-0004338,J000110.18-000434.6,0.29252029523,0.0274,-0.07639817491,0.0198,15.7099,0.0311,42.194,0.038,-64.748,0.019,7.981884,0.764665,-3.02,0.16,J2016.0,5793.3003,5743.0,5894.0,1.3951235,1.347859,1.419669,1.9752595,1.9633069,1.9872122,,0.0,0.0,0
0.4571711894795,65.4555699380329,HD224919,432197890377254528,432197890377254528,-1,224919,418988281,00014974+6527200,J000149.74+652720.6,0.45736153541,0.0117,65.45580468513,0.012,15.3692,0.0151,17.791,0.015,52.818,0.016,7.535949,0.631671,-29.18,0.15,J2016.0,6340.8335,6278.5,6510.25,1.4473692,1.3730195,1.4762511,3.0509758,3.041751,3.0602007,,0.0,0.0,0
0.5431196625046,27.0818330846237,85 Peg,2853900872207150720,2853900872207888768,-1,224930,611726941,--,--,0.54669605643,0.2633,27.07764869795,0.2213,81.179,0.449,716.46,0.452,-941.497,0.242,5.595876,0.929018,-35.57,0.35,J2016.0,5570.0,5410.6665,6035.0,0.93720734,0.7983467,0.9932179,0.7617051,0.75543445,0.76797575,true,71600.0,255.58364579917864,7
0.6965612550145,2.1301527341538,HD225028A,2739059700184753280,2739059700184753280,-1,225028A,611064061,--,--,0.69683851342,0.0193,2.12973783087,0.0154,23.3713,0.0233,62.34,0.028,-93.353,0.015,7.213268,0.799183,-2.54,0.19,J2016.0,5673.555,5333.675,6196.967,1.4055437,1.1781391,1.5903834,1.8441851,1.8386022,1.849768,,0.0,0.0,0
0.8580574171151,20.6655286255845,HD225097,2846469822870413696,2846469822870413696,276,225097,238301314,00032593+2039561,J000325.97+203954.3,0.85833620764,0.144,20.66483347784,0.185,14.5756,0.225,58.692,0.187,-156.408,0.236,7.341593,0.79231,-31.51,0.4,J2016.0,5847.0,5717.5,5932.0,1.8799559,1.8264658,1.9660815,3.7215376,3.6197815,3.8232937,,0.0,0.0,0
1.2346725514673,23.2696293916025,HD225261,2848150288954631296,2848150288954631296,400,225261,258866681,00045634+2316106,--,1.23652388841,0.0188,23.26959631737,0.0219,37.9297,0.0258,382.667,0.024,-7.444,0.029,7.624367,0.940167,7.51,0.16,J2016.0,5313.75,5286.5,5386.0,0.8244969,0.802525,0.8330188,0.48828933,0.48727214,0.4893065,true,71700.0,133.9328189802634,2
1.2396804613202,45.6737925276659,Gaia EDR3 386648181647057152,386648181647057152,386648181647057152,404,--,439946040,00045752+4540257,J000457.52+454025.5,1.23979666771,0.0154,45.67350137279,0.012,13.6393,0.0222,18.27,0.019,-65.51,0.016,7.438472,0.694592,-17.15,0.16,J2016.0,6140.0,6086.0,6194.0,1.8144845,1.7829846,1.8468266,4.215749,4.194839,4.2366586,true,71600.0,261.48613729985766,5


In [17]:
table.write('Gr8stars_GaiaeDR3_formatted.csv', overwrite=True)